In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from datasets import load_dataset

### 加载数据集

In [ ]:
d = load_dataset(path='tiny_shakespeare')

In [ ]:
text = d['train']['text'][0] + d['test']['text'][0] + d['validation']['text'][0]
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)
with open('./input.txt', 'w') as f:
    f.write(text)

### 构造字母级别的词典

In [ ]:
# tokenize
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda x: [stoi[c] for c in x]
decode = lambda l: ''.join(itos[i] for i in l)

print(encode('hii there'))
print(decode(encode('hii there')))

### 将数据集转换成Tensor

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

In [ ]:
# 划分训练集和验证集
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} the target is {target}')

In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if(split == 'train') else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y
    
xb, yb = get_batch('train')
print('inputs: ')
print(xb.shape)
print(xb)
print('targets: ')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when input is {context.tolist()} the target: {target}')

### **二元**语言模型

In [ ]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx)    # (B,T,C)   # 取table的第idx行

        if(targets is None):
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only the last time step
            logits = logits[:, -1, :]   # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running squence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
idx = torch.zeros(1,1, dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

In [ ]:
# 训练
batch_size = 32
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

for steps in range(10):
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    print(loss.item())

In [ ]:
print(decode(m.generate(idx, max_new_tokens=500)[0].tolist()))

### 在self-attention中的数学trick

In [ ]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2   # batch, time, channels
x = torch.randn(B,T,C)
x.shape

#### We want x[b,t] = $mean_{i\leq t}$ x[b,i]

In [ ]:
# version 1
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, axis=0)

In [ ]:
# version 2
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdims=True)
xbow2 = wei @ x # (T, T) @ (B, T, C) --广播--> (T, T) @ (B, T, C) ----> (B, T, C)
print('xbow and xbow2 are the same:', torch.allclose(xbow, xbow2))
print(xbow[0], '\n', xbow2[0])

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('b=')
print(b)
print('c=')
print(c)

### 自注意力机制

In [ ]:
# version 4: self-attention
torch.manual_seed(1337)
B,T,C = 4,8,32  # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)  # (B, T, 16)
q = query(x)    # (B, T, 16)
v = value(x)    # (B, T, 16)
wei = q @ k.transpose(-2, -1)   # (B, T, 16) @ (B, 16, T) ----> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
out = wei @ v
# out = wei @ x

out.shape

#### 实际论文中的公式中: $$Attention(Q,K,V) = softmax({QK^T \over \sqrt{d_k}})V$$

In [ ]:
# 除以根号dk(dimention-k)的原因: softmax容易使大的值更大, 小的值更小, 向两极分化, 因此除以根号dk保准方差在1左右
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1)
print(f'方差-k: {k.var()}, 方差-q: {q.var()}, 方差-wei: {wei.var()}')
print('--------------------------------------------------------')
wei = wei * head_size**-0.5
print(f'方差-k: {k.var()}, 方差-q: {q.var()}, 方差-wei: {wei.var()}')

### 优化方式——归一化

In [ ]:
class BatchNorm1d:

    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        # parameters (trained with backprop)
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        # buffers (trained with a running 'momentum update)
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)

    def __call__(self, x):
        # caculate the foward pass
        if(self.training):
            xmean = x.mean(0, keepdim=True) # batch mean
            xvar = x.var(0, keepdim=True)   # batch variance
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)    # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        # update the buffers
        if(self.training):
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]

torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32, 100)    # batch size 32 of 100-dimensional vectors
x = module(x)
print(x.shape)
print(x[:,0].mean(), x[:,0].std())

In [ ]:
class LayerNorm1d:

    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

    def __call__(self, x):
        xmean = x.mean(1, keepdim=True) # layer mean
        xvar = x.var(1, keepdim=True)   # layer variance
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)    # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100)    # batch size 32 of 100-dimensional vectors
x = module(x)
print(x.shape)
print(x[0,:].mean(), x[0,:].std())